In [1]:
data_path = '../dataset'

In [2]:
import os

In [3]:
for path in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, path)):
        print(os.path.join(data_path, path))

../dataset/test
../dataset/train
../dataset/val


In [4]:
train_data_path = os.path.join(data_path, 'train')
val_data_path = os.path.join(data_path, 'val')
test_data_path = os.path.join(data_path, 'test')

In [5]:
train_classes = dict()

for path in sorted(os.listdir(train_data_path)):
    if os.path.isdir(os.path.join(train_data_path, path)):
        train_classes.setdefault(len(train_classes), path)
        
train_classes

{0: 'bart_simpson',
 1: 'charles_montgomery_burns',
 2: 'homer_simpson',
 3: 'krusty_the_clown',
 4: 'lisa_simpson',
 5: 'marge_simpson',
 6: 'milhouse_van_houten',
 7: 'moe_szyslak',
 8: 'ned_flanders',
 9: 'principal_skinner'}

In [6]:
val_classes = dict()

for path in sorted(os.listdir(val_data_path)):
    if os.path.isdir(os.path.join(val_data_path, path)):
        val_classes.setdefault(len(val_classes), path)
        
val_classes

{0: 'bart_simpson',
 1: 'charles_montgomery_burns',
 2: 'homer_simpson',
 3: 'krusty_the_clown',
 4: 'lisa_simpson',
 5: 'marge_simpson',
 6: 'milhouse_van_houten',
 7: 'moe_szyslak',
 8: 'ned_flanders',
 9: 'principal_skinner'}

In [7]:
test_classes = dict()

for path in sorted(os.listdir(test_data_path)):
    if os.path.isdir(os.path.join(test_data_path, path)):
        test_classes.setdefault(len(test_classes), path)
        
test_classes

{0: 'bart_simpson',
 1: 'charles_montgomery_burns',
 2: 'homer_simpson',
 3: 'krusty_the_clown',
 4: 'lisa_simpson',
 5: 'marge_simpson',
 6: 'milhouse_van_houten',
 7: 'moe_szyslak',
 8: 'ned_flanders',
 9: 'principal_skinner'}

In [8]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
torch.cuda.get_device_name(0)

'GeForce GTX 1070'

In [10]:
torch.cuda.empty_cache()

In [11]:
from torchvision import transforms as T

In [12]:
train_transform = T.Compose([
    T.Resize((32,32)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [13]:
val_transform = T.Compose([
    T.Resize((32,32)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
test_transform = T.Compose([
    T.Resize((32,32)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
from torchvision.datasets import ImageFolder

In [16]:
train_dataset = ImageFolder(
    root=train_data_path,
    transform=train_transform
)

In [17]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 6500
    Root location: ../dataset/train
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [18]:
val_dataset = ImageFolder(
    root=val_data_path,
    transform=val_transform
)

In [19]:
val_dataset

Dataset ImageFolder
    Number of datapoints: 1500
    Root location: ../dataset/val
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [20]:
test_dataset = ImageFolder(
    root=test_data_path,
    transform=test_transform
)

In [21]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 2000
    Root location: ../dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [22]:
from torch.utils.data import DataLoader

In [23]:
BATCH_SIZE = 32

In [24]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [25]:
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [26]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [28]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1) # 32 x 32
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # 32 x 32
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1) # 16 x 16
        self.dropout = nn.Dropout(.2)
        self.fc1 = nn.Linear(16*16*32, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [29]:
model = SimpleNet()

In [30]:
model

SimpleNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [31]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
device

'cuda'

In [32]:
model.to(device)

SimpleNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [33]:
import torch.optim as optim

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [35]:
for epoch in range(10):
    print(f"\nEpoch {epoch}")
    running_loss = .0
    running_corrects = .0
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        res = torch.sum(preds == labels)
        running_corrects += res
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
    print(f"Train Loss: {epoch_loss}, Train Acc: {epoch_acc}")


Epoch 0
Train Loss: 1.7278616738686194, Train Acc: 0.4073846153846154

Epoch 1
Train Loss: 1.1892748608222374, Train Acc: 0.6072307692307692

Epoch 2
Train Loss: 0.9579393722827618, Train Acc: 0.686

Epoch 3
Train Loss: 0.752007887326754, Train Acc: 0.7504615384615385

Epoch 4
Train Loss: 0.5732069048514733, Train Acc: 0.8086153846153846

Epoch 5
Train Loss: 0.4357081503134507, Train Acc: 0.8483076923076923

Epoch 6
Train Loss: 0.3358047257661819, Train Acc: 0.8852307692307693

Epoch 7
Train Loss: 0.2523014120046909, Train Acc: 0.9153846153846155

Epoch 8
Train Loss: 0.20183718724434194, Train Acc: 0.9310769230769231

Epoch 9
Train Loss: 0.1677440095956509, Train Acc: 0.9433846153846154


In [36]:
model

SimpleNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [39]:
%%time

for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels)
    
loss = running_loss / len(test_dataset)
acc = running_corrects.double() / len(test_dataset)

CPU times: user 3.59 s, sys: 35.9 ms, total: 3.62 s
Wall time: 3.62 s


In [40]:
print(f"Test Loss: {epoch_loss}, Test Accuracy: {epoch_acc}")

Test Loss: 0.1677440095956509, Test Accuracy: 0.9433846153846154
